Code for training 2D Classifier (Not used in final project)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.optim as optim
import numpy
import cv2
from imutils import paths

Check for CUDA

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')
  
print(device)

In [ ]:
model = models.resnet18(pretrained=True)
num_classes = 10

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
model.to(device)

Training the Model

In [ ]:
transform_img = transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 320))])

train_path = "/content/drive/MyDrive/EE381K/UCF11/ucf_action/training"
train_dataset = datasets.ImageFolder(train_path, transform=transform_img)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

print('Starting training')
model.train()

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model.forward(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
            torch.save(model.state_dict(), '/content/chkp/check{}.pt'.format((i + 1) / 1000))
        break

print('Finished Training')

# saving trained model
PATH = r'/content/classfier.pt'
torch.save(model.state_dict(), PATH)
print('saved model')

Loading Saved Model

In [ ]:
model.load_state_dict(torch.load(PATH))

Testing the Model

In [ ]:
print('Start testing')
model.eval()

transform_img = transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 320)), transforms.RandomHorizontalFlip(1.0)])
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/EE381K/UCF11/ucf_action/validation', transform=transform_img)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

correct = 0
total = 0
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model.forward(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('correct: {}, total: {}'.format(correct, total))
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))